In [1]:
from scipy.integrate import odeint
from scipy import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import bqplot as bq
from ipywidgets import interact, interactive, Layout, Box, HBox, VBox, Label
import ipywidgets as ipyw
from IPython.display import display

%matplotlib inline

In [ ]:
## Time interval of the simulations
t = arange(0.0, 50.0, 0.01)
m = lorenz()

## Solving the diffential equations
sol = odeint(m.dXdt, m.X0, t)

In [ ]:
## Plot solution timeseries
fig, ax = plt.subplots()
for i, v in enumerate(m.label):
    figY = plt.plot(t, sol[:,i], label=v)

plt.xlabel('Time (ms)')
plt.legend(frameon=False)
plt.show()

In [ ]:
## The butterfly!
fig = plt.figure()
ax = fig.gca(projection='3d')

x, y, z = sol.T
ax.plot(x, y, z)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()

In [2]:
### Lorenz Model Class
class lorenz:
    ## Must be in same order as dXdt return
    label = ['x', 'y', 'z']
    nVar = len(label)

    ## Parameters
    p = {
    's': 10,
    'r': 28,
    'b': 2.667
    }

    ## Get initial values for the system
    def __init__(self, X0 = [0.3, 0.2, 0.1], T=arange(0.0, 50.0, 0.01)):
        self.T = T
        self.name = self.__class__.__name__
        if self.nVar != len(X0):
            print 'ERROR:', self.nVar, 'initial values required for:', self.name
        else: 
            self.X0 = X0

    ## Define the dX/dt for the system
    def dXdt(self, X, t):
        x, y, z = X

        dxdt = self.p['s']*(y - x)
        dydt = self.p['r']*x - y - x*z
        dzdt = x*y - self.p['b']*z

        return dxdt, dydt, dzdt

In [53]:
### Hodgekin-Huxley Model Class
class HH:
    ## Must be in same order as dXdt return
    label = ['V', 'm', 'h', 'n']
    nVar = len(label)

    ## Parameters
    p = {
    'C_m' : 1.0,
    'g_Na': 120.0,
    'g_K' : 36.0,
    'g_L' : 0.3,
    'E_Na': 50.0,
    'E_K' : -77.0,
    'E_L' : -54.387
    }

    ## Get initial values for the system
    def __init__(self, X0 = [-65, 0.05, 0.6, 0.32], T=arange(0.0, 50.0, 0.01)):
        self.T = T
        self.name = self.__class__.__name__
        if self.nVar != len(X0):
            print 'ERROR:', self.nVar, 'initial values required for:', self.name
        else: 
            self.X0 = X0
            
            
    ## External current (step function)
    def I_ext(self, t): return 10*floor(t/25)
         
    # Channel gating variables (ms)
    def alpha_m(self, V):  return 0.1*(V+40.0)/(1.0 - exp(-(V+40.0) / 10.0))
    def beta_m(self, V):   return 4.0*exp(-(V+65.0) / 18.0)
    def alpha_h(self, V):  return 0.07*exp(-(V+65.0) / 20.0)
    def beta_h(self, V):   return 1.0/(1.0 + exp(-(V+35.0) / 10.0))
    def alpha_n(self, V):  return 0.01*(V+55.0)/(1.0 - exp(-(V+55.0) / 10.0))
    def beta_n(self, V):   return 0.125*exp(-(V+65) / 80.0)

    # Membrane current (in uA/cm^2)
    def I_Na(self, V, m, h):  return self.p['g_Na'] * m**3 * h * (V - self.p['E_Na'])
    def I_K(self, V, n):      return self.p['g_K']  * n**4 * (V - self.p['E_K'])
    def I_L(self, V):         return self.p['g_L'] * (V - self.p['E_L'])

    
    ## Define the dX/dt for the system
    def dXdt(self, X, t):
        V, m, h, n = X

        dV = self.I_ext(t) - self.I_Na(V, m, h) - self.I_K(V, n) - self.I_L(V)/ self.p['C_m']
        dm = self.alpha_m(V)*(1.0-m) - self.beta_m(V)*m
        dh = self.alpha_h(V)*(1.0-h) - self.beta_h(V)*h
        dn = self.alpha_n(V)*(1.0-n) - self.beta_n(V)*n

        return dV, dm, dh, dn

In [57]:
class slider():
    cpalette = ['#EE224A', '#22AF4B', '#4CB5F5', '#FF5C00', 
                '#08B9A5', '#15AB00', '#881EE4', '#5C6BC0']
    
    def __init__(self, model):
        self.model = model
        self.lines = {}
        self.param = {}
        self.cb = {}
        self.colors = {}
        
        self._solve()
        self._curveColors()
        self._plot()
        
        
    def _curveColors(self):
        for i,v in enumerate(self.model.label):
            self.colors.update({v: self.cpalette[i]})
        
    def _solve(self):
        sol = odeint(self.model.dXdt, self.model.X0, self.model.T)
        
        self.sol = {}
        for i,var in enumerate(self.model.label):
            self.sol.update({var: sol[:,i]})
    
    def _getFig(self):
        for v in self.model.label:
            x_sc = bq.LinearScale()
            y_sc = bq.LinearScale()

            x_ax = bq.Axis(label='Time', scale=x_sc, tick_format='0.0f', color='black', grid_lines='solid', grid_color='#ddd')
            y_ax = bq.Axis(label=v,      scale=y_sc, tick_format='0.2f', color='black', grid_lines='solid', grid_color='#ddd', orientation='vertical')

            # Generate a line (but does not plot it)
            l = bq.Lines(x=self.model.T, y=self.sol[v], colors=[self.colors[v]],
                              scales={'x': x_sc, 'y': y_sc})
        
            self.lines.update({v: l})
          
        fig = bq.Figure(axes=[x_ax, y_ax], marks=self.lines.values(),
                       fig_margin={'top':10, 'bottom':0, 'left':60, 'right':10},
                       max_aspect_ratio=3, min_aspect_ratio=2.5)
        fig.layout.width = '100%'
        return fig
        
    def _plot(self):
        self._solve()
        fig = self._getFig()
        
        display(fig)
        
    def _updateFig(self):        
        self._solve()
        for v in self.model.label:
            self.lines[v].y = self.sol[v]
    
    ### Parameter handling
    def _paramUpdate(self, change):
        self.model.p[change['owner'].description] = change['new']
        self._updateFig()
            
    def paramSlider(self):
        for k,v in self.model.p.items():
            print v
            crude = ipyw.IntSlider(
                min = -5, max = 10,
                description=k,
                value = floor(log10(v)), 
                continuous_update = False
            )

            fine = ipyw.FloatSlider(
                min = 10**floor(log10(v)),
                max = 10**(floor(log10(v))+1),
                description=k, 
                value = v,
                continuous_update = False
            )
            
            #txt = ipyw.FloatText(value=v, description=k, layout=Layout(width='80px'))
        
            #ipyw.jslink((fine, 'value'), (txt, 'value'))
            crude.observe(self._updateRange, names='value')
            fine.observe(self._paramUpdate,  names='value')
        
            p = [crude, fine]#, txt]
            self.param.update({k: p})
        
        box_layout = Layout(display='flex-start',
                    flex_flow='row',
                    align_items='flex-start',
                    align_content='flex-start',
                    width='100%')
        
        for k, v in self.param.items():
            box = Box(children=v, layout=box_layout)
            display(box, layout=Layout(align_items='flex-start'))
            
            
    def _updateRange(self, c):
        k = c['owner'].description
        try:
            self.param[k][1].max = 10**(c['new']+1)
            self.param[k][1].min = 10**c['new']
        except:
            self.param[k][1].min = 10**c['new']
            self.param[k][1].max = 10**(c['new']+1)
        self.param[k][1].value = self.param[k][1].min
        self.param[k][1].step = 1 if c['new']>0 else 10**(c['new']-1)
        
    ### Toggle curve display
    def _toggleCurve(self, b):
        v = not self.lines[b.description].visible
        if self.lines[b.description].visible:
            self.cb[b.description].style.button_color = '#ddd'
            self.lines[b.description].visible = v
        else:
            self.cb[b.description].style.button_color = self.colors[b.description]
            self.lines[b.description].visible = v
        
        
    def _getToggleButton(self, v):
        cb = ipyw.Button(
            value = self.lines[v].visible,
            description = v,
            style = ipyw.ButtonStyle(button_color=self.colors[v]),
            layout = Layout(width='100px')
        )
        return cb
        
    def toggleButton(self):
        for v in self.model.label:
            self.cb.update({v: self._getToggleButton(v)})
            
        display(VBox(self.cb.values()))
        
        for cb in self.cb.values():
            cb.on_click(self._toggleCurve)

In [48]:
X0 = [0.3, 0.2, 0.1]
tmin, tmax, dt = 0, 30, 0.01
T = arange(tmin,tmax,dt)
m = slider(model=lorenz(X0=X0, T=T))

m.paramSlider1()
m.toggleButton()

RmlndXJlKGF4ZXM9W0F4aXMoY29sb3I9J2JsYWNrJywgZ3JpZF9jb2xvcj0nI2RkZCcsIGxhYmVsPXUnVGltZScsIHNjYWxlPUxpbmVhclNjYWxlKCksIHRpY2tfZm9ybWF0PXUnMC4wZicpLCDigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MSwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUncycsIG1heD0xMCwgbWluPS01KSwgRmxvYXRTbGlkZXIodmFsdWXigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MSwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUncicsIG1heD0xMCwgbWluPS01KSwgRmxvYXRTbGlkZXIodmFsdWXigKY=


Qm94KGNoaWxkcmVuPShJbnRTbGlkZXIodmFsdWU9MCwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUnYicsIG1heD0xMCwgbWluPS01KSwgRmxvYXRTbGlkZXIodmFsdWXigKY=


VkJveChjaGlsZHJlbj0oQnV0dG9uKGRlc2NyaXB0aW9uPXUneScsIGxheW91dD1MYXlvdXQod2lkdGg9dScxMDBweCcpLCBzdHlsZT1CdXR0b25TdHlsZShidXR0b25fY29sb3I9JyMyMkFGNELigKY=


In [58]:
m = slider(model=HH())

m.paramSlider1()
m.toggleButton()

RmlndXJlKGF4ZXM9W0F4aXMoY29sb3I9J2JsYWNrJywgZ3JpZF9jb2xvcj0nI2RkZCcsIGxhYmVsPXUnVGltZScsIHNjYWxlPUxpbmVhclNjYWxlKCksIHRpY2tfZm9ybWF0PXUnMC4wZicpLCDigKY=


1.0
0.3
120.0
36.0
-77.0


TypeError: ufunc 'floor' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''